In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%load_ext dotenv

In [3]:
import os

In [4]:
!pwd
!ls -l ./

/mnt/datadrive01/prj/transfer_learning/outsideAML/code
total 284
-rw-rw-r-- 1 loginVM_001 loginVM_001 265312 Sep 25 03:31 00_create_docker_image.html
-rw-rw-r-- 1 loginVM_001 loginVM_001  14236 Sep 25 03:32 00_create_docker_image.ipynb
-rw-rw-r-- 1 loginVM_001 loginVM_001   6602 Sep 25 01:27 edit_python_files.ipynb


In [5]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "/transfer_learning_analysis"

Overwriting .env


In [6]:
!ls -l ./../

total 8
drwxrwxrwx 3 root root 4096 Sep 25 03:32 code
drwxrwxrwx 2 root root 4096 Sep 25 01:49 docker


In [7]:
%%writefile ./../docker/dockerfile

FROM debian:9.5

MAINTAINER George Iordanescu <ghiordan@microsoft.com>

# Install system packages
RUN apt-get update && apt-get install -y --no-install-recommends \
      apt-utils \
      apt-transport-https \ 
      ca-certificates \
      curl \
      bzip2 \
      g++ \
      git \
      graphviz \
      wget \
      ssh \
      gnupg \
      gnupg2 \
      rsync && \
    rm -rf /var/lib/apt/lists/*

# Install conda
ENV CONDA_DIR /opt/conda
ENV PATH $CONDA_DIR/bin:$PATH

RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linux-x86_64.sh" | sha256sum -c - && \
    /bin/bash /Miniconda3-4.2.12-Linux-x86_64.sh -f -b -p $CONDA_DIR && \
    rm Miniconda3-4.2.12-Linux-x86_64.sh && \
    echo export PATH=$CONDA_DIR/bin:'$PATH' > /etc/profile.d/conda.sh
        
#https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux
# https://docs.microsoft.com/en-us/dotnet/core/linux-prerequisites?tabs=netcore2x
#https://www.microsoft.com/net/download/linux-package-manager/debian9/runtime-2.1.2
RUN wget --no-check-certificate -qO- https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.asc.gpg && \
    mv microsoft.asc.gpg /etc/apt/trusted.gpg.d/ && \
    wget -q https://packages.microsoft.com/config/debian/9/prod.list && \
    mv prod.list /etc/apt/sources.list.d/microsoft-prod.list && \
    chown root:root /etc/apt/trusted.gpg.d/microsoft.asc.gpg && \
    chown root:root /etc/apt/sources.list.d/microsoft-prod.list  && \
    apt-get update && \
    apt-get install -y --no-install-recommends aspnetcore-runtime-2.1 && \
    mkdir /tmp/azcopy && \
    wget -O /tmp/azcopy/azcopy.tar.gz https://aka.ms/downloadazcopyprlinux &&  \
    tar -xf /tmp/azcopy/azcopy.tar.gz -C /tmp/azcopy &&  \
    /tmp/azcopy/install.sh && \
    rm -rf /tmp/azcopy

# Install Python packages and keras
ENV NB_USER tlvisuser
ENV NB_UID 1000

RUN useradd -m -s /bin/bash -N -u $NB_UID $NB_USER && \
    chown $NB_USER $CONDA_DIR -R && \
    mkdir -p /src && \
    chown $NB_USER /src

USER $NB_USER

ARG python_version=3.6

# tensorflow-gpu 1.10.1 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.15.2 which is incompatible.
RUN conda install -y python=${python_version} && \
    pip install --upgrade pip && \
    pip install \
      sklearn_pandas \
      scikit-image \
      tensorflow \
      tensorflow-tensorboard \
      numpy==1.14.5 && \
    conda install \
      glob2 \
      h5py \
      matplotlib \
      mkl \
      nose \
      notebook \
      opencv \
      pandas \
      requests \
      scikit-learn \
      scipy \
      six \
      tqdm && \
    conda install -c conda-forge imageio && \
    git clone git://github.com/keras-team/keras.git /src && pip install -e /src[tests] && \
    pip install git+git://github.com/keras-team/keras.git && \
    pip install git+https://www.github.com/keras-team/keras-contrib.git && \
    conda clean -yt     

ENV PYTHONPATH='/src/:$PYTHONPATH'

WORKDIR /src

EXPOSE 8888

# CMD jupyter notebook --port=8888 --ip=0.0.0.0


Overwriting ./../docker/dockerfile


In [8]:
%dotenv
docker_image_name = os.getenv('docker_login') + os.getenv('image_tag') + ':1.0.0'
docker_file_location = os.path.join(*(['.', '..','docker', 'dockerfile']))
working_path = '.'

docker_image_name
docker_file_location

'georgedockeraccount/transfer_learning_analysis:1.0.0'

'./../docker/dockerfile'

In [9]:
!echo docker build -t $docker_image_name -f $docker_file_location $working_path
!docker build -t $docker_image_name -f $docker_file_location $working_path

docker build -t georgedockeraccount/transfer_learning_analysis:1.0.0 -f ./../docker/dockerfile .
Sending build context to Docker daemon  309.3kB
Step 1/16 : FROM debian:9.5
 ---> f2aae6ff5d89
Step 2/16 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> ac222101e003
Step 3/16 : RUN apt-get update && apt-get install -y --no-install-recommends       apt-utils       apt-transport-https       ca-certificates       curl       bzip2       g++       git       graphviz       wget       ssh       gnupg       gnupg2       rsync &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 4a92d6615c5d
Step 4/16 : ENV CONDA_DIR /opt/conda
 ---> Using cache
 ---> d7b3a1ff7b4f
Step 5/16 : ENV PATH $CONDA_DIR/bin:$PATH
 ---> Using cache
 ---> bc5384a3eb21
Step 6/16 : RUN wget --quiet --no-check-certificate https://repo.continuum.io/miniconda/Miniconda3-4.2.12-Linux-x86_64.sh &&     echo "c59b3dd3cad550ac7596e0d599b91e75d88826db132e4146030ef471bb434e9a *Miniconda3-4.2.12-Linu

In [10]:
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/transfer_learning_analysis]

aa3d5292: Preparing 
35c623c8: Preparing 
379a1fcf: Preparing 
8a9c05e5: Preparing 
c39c2b02: Preparing 
f0b6fef8: Layer already exists 1.0.0: digest: sha256:df1de7fc7dfb1ecc9f2084657a7603144b495eb3cb810bedf0553194bbf069c8 size: 1592


In [11]:
!echo docker run -i -t -p 10003:8888 -v '$(pwd)':/local_dir:rw $docker_image_name /bin/bash -c '"/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip='*' --port=8888 --no-browser --allow-root"'

docker run -i -t -p 10003:8888 -v $(pwd):/local_dir:rw georgedockeraccount/transfer_learning_analysis:1.0.0 /bin/bash -c "/opt/conda/bin/jupyter notebook --notebook-dir=/local_dir --ip=* --port=8888 --no-browser --allow-root"


In [12]:
!jupyter nbconvert --to html 00_create_docker_image.ipynb

[NbConvertApp] Converting notebook 00_create_docker_image.ipynb to html
[NbConvertApp] Writing 266194 bytes to 00_create_docker_image.html
